# Задание 1. Градационные преобразования.

**Автор**: 22.М04, Винник Екатерина Петровна

**Задание**: реализовать градационные преобразования: 
   - [+] негатив 
   - [+] линейные 
   - [+] логарифмические
   - [+] степенные
   - [+] кусочно-линейные.

Работа выполнена на языке Python в интерактивном блокноте Jupyter Notebook.

## Кусочно-линейные градационные преобразования

Одним из простейших случаев использования кусочно-линейных функций является преобразование, усиливающее контрасты. Вид преобразования задают положения точек $(r_1, s_1)$ $(r_2, s_2)$.

<p>
    <img src="./pwl.png" alt="pl"/>
    <br>
    <div align="center">
        <em>Вид функции преобразования</em>
    </div>
</p>

Рассмотрим реализации преобразования, использующие кусочно-линейные функции, т.е., где:
- $r_1$, $r_2$, $s_1$, $s_2$ -- произвольные (случай кусочно-линейной функции)
- $r_1 = r_2$, $s_1 = 0$, $s_2 = L - 1$ (случай пороговой функции)

### Кусочно-линейная функция 

На вход функции *pwl_transform*, реализующей преобразование, использующее кусочно-линейную функцию, передается изображение и точки $(r_1, s_1)$ $(r_2, s_2)$, задающие вид функции преобразования.

In [1]:
import numpy as np


def pwl_transform(img, r1=.5, r2=.5, s1=0., s2=1.): # the values are normalized to the range [0, 1]
    
    # normalize the image
    img_norm = img / 255.
    
    k1 = s1 / r1
    k2 = (s2 - s1) / (r2 - r1)
    b2 = (r2*s1 - r1*s2) / (r2 - r1)
    k3 = (1 - s2) / (1 - r2)
    b3 = (s2 - r2) / (1 - r2)
    
    img_new_norm = img_norm
    img_1_idx = img_norm < r1
    img_temp = np.zeros_like(img_norm, dtype=np.float32)
    img_temp[img_norm >= r1] = 1.
    img_temp[img_norm >= r2] = 0.
    img_2_idx = img_temp == 1.
    img_3_idx = img_norm >= r2
    
    img_new_norm[img_1_idx] = k1 * img_norm[img_1_idx]
    img_new_norm[img_2_idx] = k2 * img_norm[img_2_idx] + b2
    img_new_norm[img_3_idx] = k3 * img_norm[img_3_idx] + b3
    
    img_new = img_new_norm * 255.
    
    return img_new

#### Пример применения преобразования, использующего кусочно-линейную функцию

In [2]:
import numpy as np
from PIL import Image

# From R. Gonzales, R. Woods "Digital Image Processing"
img_1 = Image.open('./eins.tif')

In [3]:
img_1_np = np.array(img_1)
print('Picture dimensions:')
print(img_1_np.shape)

Picture dimensions:
(600, 490)


In [4]:
# arbitrary values
img_1_np_pwl = pwl_transform(img_1_np, .35, .65, .15, .85)
img_1_pwl = Image.fromarray(img_1_np_pwl.astype(np.uint8))

In [5]:
img_1 = Image.fromarray(img_1_np.astype(np.uint8))
img_1_pwl.save('./eins_pwl.png')
img_1.save('./eins.png')

<p>
    <br>
    <img src="./eins.png" align="left"/> <img src="./eins_pwl.png" align="left"/>
    <br>
    <br>
    <div align="center">
        <em>Фотография Эйнштейна до и после преобразования, использующего кусочно-линейную функцию</em>
    </div>
</p>

Как видим, в результате применения градационного преобразования с использованием кусочно-линейной функции контрастность изображения действительно повысилась.

### Пороговая функция

На вход функции *trsh_transform* передается изображение, а также параметр $m$ -- среднее значение яркостей на изображении.

In [6]:
import numpy as np


def trsh_transform(img, m):
    
    # use the pwl_transform function to obtain results
    img_new = np.zeros_like(img)
    img_new[img > m] = 255
    
    return img_new

#### Пример применения преобразования, использующего пороговую функцию

In [7]:
# From R. Gonzales, R. Woods "Digital Image Processing"
img_3 = Image.open('./dollar.tif')

In [8]:
img_3_np = np.array(img_3)
print('Picture dimensions:')
print(img_3_np.shape)

Picture dimensions:
(500, 1192)


In [9]:
# using the threshold transform
img_3_mean = np.mean(img_3_np)
img_3_np_trsh = trsh_transform(img_3_np, img_3_mean)
img_3_trsh = Image.fromarray(img_3_np_trsh.astype(np.uint8))

In [10]:
img_3_trsh.save('./dollar_trsh.png')
img_3.save('./dollar.png')

<p>
    <br>
    <img src="./dollar.png" align="left" style="width:50%"/> <img src="./dollar_trsh.png" align="left" style="width:50%"/>
    <br>
    <br>
    <div align="center">
        <em>Фотография доллара до и после преобразования с использованием пороговой функции</em>
    </div>
</p>

Как видим, в результате применения преобразования, использующего пороговую функцию, изображение доллара стало бинарным.

## Линейные преобразования

При значениях точек $(r_1, s_1)=(r_{min},0)$ и $(r_2, s_2)=(r_{max}, L-1)$ (где $r_{min}$ и $r_{max}$ -- минимальная и максимальная яркости изображения), задающих вид кусочно-линейного преобразования, функция преобразования *линейно* растягивает исходный диапазон яркостей в полный диапазон яркостей $[0, L-1]$.

Поэтому можем воспользоваться ранее написанной функцией *pwl_transform* и вызвать ее с параметрами $(r_{min}, 0)$, $(r_{max}, 255)$, предварительно посчитав минимальную и максимальную яркости изображения.


Посчитаем минимальную и максимальную яркости изображения.

In [11]:
img_2 = Image.open('./pic.tif')

In [12]:
img_2_np = np.array(img_2)

In [13]:
img_2_min = img_2_np.min(axis=(0, 1))
img_2_max = img_2_np.max(axis=(0, 1))
print('Min values:')
print(img_2_min)
print('Max values:')
print(img_2_max)

img_2_np = pwl_transform(img_2_np, img_2_min / 255, img_2_max / 255, 0., 1.)

Min values:
91
Max values:
138


In [14]:
img_2_lin = Image.fromarray(img_2_np.astype(np.uint8))
img_2_lin.save('./pic_linear.png')
img_2.save('./pic.png')

<p>
    <br>
    <img src="./pic.png" align="left" style="width:50%"/> <img src="./pic_linear.png" align="left" style="width:50%"/>
    <br>
    <br>
    <div align="center">
        <em>До и после линейного преобразования</em>
    </div>
</p>

Как видим, примененное линейное преобразование сделало изображение светлее, а также усилило его контрастность.

## Негатив

Принимая во внимание, что негативное преобразование для яркостей в диапазоне $[0, L-1]$ задается выражением $s=L-1-r$, можем реализовать функцию негативного преобразования *neg_transform*, принимающую на вход изображение.

In [15]:
def neg_transform(img):
   
    img_new = 255 - img
    
    return img_new

#### Пример применения преобразования, использующего негатив

In [16]:
# From R. Gonzales, R. Woods "Digital Image Processing"
img_21 = Image.open('./moon.tif')

In [17]:
img_21_np = np.array(img_21)

In [18]:
print('Picture dimensions:')
print(img_21_np.shape)

Picture dimensions:
(540, 466)


In [19]:
img_21_np_neg = neg_transform(img_21_np)
img_21_neg = Image.fromarray(img_21_np_neg.astype(np.uint8))

In [20]:
img_21 = Image.fromarray(img_21_np.astype(np.uint8))
img_21_neg.save('./moon_neg.png')
img_21.save('./moon.png')

<p>
    <br>
    <img src="./moon.png" align="left" style="width:50%"/> <img src="./moon_neg.png" align="left" style="width:50%"/>
    <br>
    <br>
    <div align="center">
        <em>Луна до и после негатива</em>
    </div>
</p>

Как видим, после негативного преобразования цвета картинки инвертировались и фон стал белым.

## Степенные градационные преобразования

Степенные преобразования имеют вид $s=c*r^\gamma$, где $c$ и $\gamma$ -- положительные константы. На вход функции *power_transform* передается изображение и эти константы.

Гамма-коррекция позволяет сделать изображение светлее, что, например, часто используется для осветления снимков, сделанных на основе мониторов с электронно-лучевыми трубками.

In [21]:
import numpy as np


def power_transform(img, c=1., gamma=1.):
    
    img_norm = img / 255.
    img_norm_new = c * np.power(img_norm, gamma)
    if img_norm_new.max() > 1.:
        img_norm_new = img_norm_new / img_norm_new.max()
    img_new = img_norm_new * 255
        
    return img_new

#### Пример применения преобразования, использующего функцию гамма-коррекции

In [22]:
# Loheland (materials for the course)
img_23 = Image.open('./field.png')
# Loheland (materials for the course)
img_24 = Image.open('./snow.png')

In [23]:
img_23_np = np.array(img_23)
img_24_np = np.array(img_24)
print('Picture dimensions:')

Picture dimensions:


In [24]:
print(img_23_np.shape)
print(img_24_np.shape)

(2736, 3648, 3)
(2736, 3648, 3)


In [25]:
# Gamma correction of a photo with a field on it
c = 1.
gamma = .8
img_23_np_pow = power_transform(img_23_np, c, gamma)
img_23_pow = Image.fromarray(img_23_np_pow.astype(np.uint8))

In [26]:
# Gamma correction of a photo with a snow
c = 1.
gamma = .7
img_24_np_pow = power_transform(img_24_np, c, gamma)
img_24_pow = Image.fromarray(img_24_np_pow.astype(np.uint8))

In [27]:
img_23_pow.save('./field_pow.png')
img_24_pow.save('./snow_pow.png')

<p>
    <br>
    <img src="./field.png" align="left" style="width:50%"/><img src="./field_pow.png" align="left" style="width:50%"/>
    <img src="./snow.png" align="left" style="width:50%"/><img src="./snow_pow.png" align="left" style="width:50%"/>
    <br>
    <br>
    <div align="center">
        <em> Фотографии луга и снега до и после степенного образования при $\gamma = 0.8$ and $\gamma = 0.7$ соответственно</em>
    </div>
</p>

Как видим, в результате применения преобразования, использующего функцию гамма-коррекции, изображения действительно стали светлее.

## Логарифмические градационные преобразования

Общий вид логарифмического градационного преобразования выражается формулой $s=clog(1+r)$, где $c$ -- константа и $r >= 0$.
На вход функции *log_transform* передается изображение и константа $c$.

In [28]:
import numpy as np


def log_transform(img, c=1.):
   
    img_norm = img / 255.
    img_norm_new = c * np.log2(img_norm + 1)
    if img_norm_new.max() > 1.:
        img_norm_new = img_norm_new / img_norm_new.max()
    img_new = img_norm_new * 255
    
    return img_new

#### Пример применения логарифмического преобразования

In [29]:
# From R. Gonzales, R. Woods "Digital Image Processing"
img_22 = Image.open('./star.tif')

In [30]:
img_22_np = np.array(img_22)
print('Picture dimensions:')
print(img_22_np.shape)

Picture dimensions:
(256, 256)


In [31]:
c = 1
img_22_np_log = log_transform(img_22_np, c)
img_22_log = Image.fromarray(img_22_np_log.astype(np.uint8))

In [32]:
img_22 = Image.fromarray(img_22_np.astype(np.uint8))

In [33]:
img_22.save('./star.png')
img_22_log.save('./star_log.png')

<p>
    <br>
    <img src="./star.png" align="left" style="width:50%"/><img src="./star_log.png" align="left" style="width:50%"/>
    <br>
    <br>
    <div align="center">
        <em> Звезда до и после логарифмического преобразования</em>
    </div>
</p>

Как видим, в результате логарифмического преобразования на изображении стало видно больше деталей, которые раньше подавлялись черным цветом.